# Notes

### NULL Model
- For a Markov(1) Model, we consider a preceding base
    - Ex:  
    
        $P(A|T_{prev}) = \frac{P(TA)}{P(T)} = \frac{\frac{c(TA)}{N}}{\frac{c(T)}{N}} = \frac{c(TA)}{c(T)}$  
        
        - N is not the same in the cases of TA and T, but the difference is negligible for now
- Increasing the order of a Markov Model makes our model more informed
- Check Week 2 Overview Video 3 for more details on detailed Markov Model for 4-mer described by $k_{1}k_{2}k_{3}k_{4}$

In [3]:
import sys
import os 
import numpy as np
from fastaReader import FastAreader

# holds files
dataList = os.listdir('mdata')
print(f"{os.listdir()} \n{os.listdir('mdata')}")
print('mdata/'+dataList[0])

### SEQUENCE LIST ###
seqList = []
readData = FastAreader('mdata/'+dataList[0]).readFasta()
for line in readData:
    seqList.append(line[1])

### REVERESE COMPLIMENT FUNCTION ###
def revComp(seqStr):
    newStr = ''
    baseDict = {'A':'T', 'T':'A', 'C':'G', 'G':'C', 'N':'N'}
    for char in seqStr:
        newStr += baseDict[char]
    return newStr[::-1]

['.ipynb_checkpoints', 'Chan_Nicholas_serachForTheMissing.ipynb', 'data', 'dummm.txt', 'fastaReader.py', 'mdata', 'notes.md', 'pasgn2.ipynb', 'program.py', 'testing.ipynb', '__pycache__'] 
['Arthrospira-platensis-NIES-39.fna', 'Ecoli-UMN026.fa', 'Synechococcus7002.fna', 'Zm4-genomic.fna']
mdata/Arthrospira-platensis-NIES-39.fna


In [20]:
class SearchForMissing:
    def __init__(self, seqList, minMotif, maxMotif, zCut):
        self.seqList = seqList
        self.minMotif = minMotif
        self.maxMotif = maxMotif
        self.zCut = zCut
        
        self.N = self.genomeLen() # - self.maxMotif
        self.ctDict = self.ctDictMake()
        self.prDict = self.prDictMake()
        self.output = self.outputMake()

    def genomeLen(self):
        '''
        Takes the sum of sequence lengths from the list of sequences, seqList.
        seqList is made with the FastAreader class and has 1 fasta seq per entry.
        '''
        totLen = 0
        for seq in seqList:
            totLen += len(seq)
        return totLen

    def ctDictMake(self):
        '''
        Stores a dict of dicts
        {1:{'A':25, 'C':33,...}, ... ,8:{'AAAAGGGG':12, 'AAAATTTT':11, ...}}
        Reads over each sequence in the fasta. For each sequence, a sliding window
        moves over its respective sequence length and k-mers are counted.
        '''
        
        massDict = dict() # dictionary that holds dictionaries which hold k-mers and their respective counts
        for k in range(1, self.maxMotif+1): # Initialize dictionaries for min to max motif sizes within massDict; MAYBE START WITH 1 AND NOT MIN MOTIF
            massDict[k] = dict()
        print(massDict)
        for seq in self.seqList:
            for k in range(1, self.maxMotif+1):
                for kmer_idx in range((len(seq)-k)):
                    kmer = seq[kmer_idx:kmer_idx+k]
                    if 'N' not in kmer:
                        if kmer in massDict[k]:
                            massDict[k][kmer] += 1
                        else:
                            massDict[k][kmer] = 1
        return massDict
                                
    def prDictMake(self): # VERY DEPENDENT ON ctDict
        massDict = dict() # dictionary that holds dictionaries which hold k-mers and their respective counts
        for k in range(self.minMotif, self.maxMotif+1): # Initialize dictionaries for min to max motif sizes within massDict
            massDict[k] = dict()
#         print(massDict)
        for k in range(self.minMotif, self.maxMotif+1):
            for kmer in self.ctDict[k].keys():
                if (kmer[:-1] not in self.ctDict[k-1]) or (kmer[1:] not in self.ctDict[k-1]) or (kmer[1:-1] not in self.ctDict[k-2]): # If sub seq is not in sequence count 
                    massDict[k][kmer] = 0
                else:
                    massDict[k][kmer] = (1/self.N) * ((self.ctDict[k-1][kmer[:-1]] * self.ctDict[k-1][kmer[1:]]) / self.ctDict[k-2][kmer[1:-1]])
        return massDict

    def outputMake(self):
        finalList = []
        for k in range(self.minMotif, self.maxMotif+1):
            banned = set()
            for kmer in self.ctDict[k].keys():
                if kmer not in banned: # interesting condition, is it implied that the reverse complement of the kmer is excluded?
                    # sequenceReverse entry
                    reverseCompKmer = revComp(kmer) # K-mer reverse compliment variable
                    tmpList = sorted([kmer,reverseCompKmer]) # Temporary list used for sorting kmer and reverse kmer for output
                    sequenceReverse = f"{tmpList[0]}:{tmpList[1]}" # sequence: reverse entry for output
                    # count entry
                    if kmer != reverseCompKmer: # Case 1: k-mer is not a palindrome
                        if reverseCompKmer in self.ctDict[k]:
                            count = self.ctDict[k][kmer] + self.ctDict[k][reverseCompKmer] 
                        else:
                            count = self.ctDict[k][kmer]
                    else: # Case 2: k-mer is a palindrome
                        count = self.ctDict[k][kmer] # interesting change
#                         if kmer == "GCGATCGC":
#                             print(f'ct of GCGATCGC: {count}')
                    # expectation entry
                    if reverseCompKmer in self.ctDict[k]:
                        Pr = self.prDict[k][kmer] + self.prDict[k][reverseCompKmer] 
#                         if kmer == "GCGATCGC":
#                             print(f'pr of GCGATCGC: {Pr}')
                    else: 
                        Pr = self.prDict[k][kmer]
                    expectation = Pr * (self.N) # look at this
                    # zscore entry
                    numerator = count - expectation
                    sd = (expectation * (1 - Pr))**0.5
                    zscore = numerator/sd
                    entry = (sequenceReverse, count, expectation, zscore, k)
                    banned.add(kmer) # Mark
                    banned.add(revComp(kmer))
                    finalList.append(entry)
#         finalList.sort(key=lambda x:x[-2]) # sort by z-score
#         finalList.sort(reverse=True, key=lambda x:x[-1]) # sort by motif length
        
        return finalList

In [21]:
sfm = SearchForMissing(seqList, 3, 8, 0)

l = sfm.output
l.sort(key=lambda x:x[-2])
l.sort(reverse=True, key=lambda x:x[-1])

{1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}}


In [22]:
print(f'N = {sfm.N}')
for line in l:
    if line[-1] < 8:
        break

    print(line)
# expectation and z score for non palindromes are doubled :-(
'''
Expectation = Prob * Genome Length

Prob = Pr(GCGATCGC) = (1/N) * (count(GCGATCG) * count(CGATCGC))/count(CGATCG)

OR if kmer's reverse complement is different

Prob = Pr(CGATCGCC) + Pr(GGCGATCG)

Z = (observed - expectation)/sd

sd = sqrt(N*P * (1-P))
'''

N = 6788435
('GCGATCGC:GCGATCGC', 7683, 15876.382478446401, -65.10223493467082, 8)
('AATTAATT:AATTAATT', 558, 1290.1022353145038, -20.38454117072713, 8)
('AAAATTTT:AAAATTTT', 501, 1092.5187558906694, -17.89736186597799, 8)
('TTAATTAA:TTAATTAA', 537, 1102.20874134225, -17.02598335646839, 8)
('AAATATTT:AAATATTT', 365, 852.584289496911, -16.699686369886066, 8)
('ATAATTAT:ATAATTAT', 364, 840.3257702412229, -16.432631605479443, 8)
('CAAATTTG:CAAATTTG', 245, 579.1215834118756, -13.88475372293913, 8)
('ACAATTGT:ACAATTGT', 115, 375.3628180039139, -13.438950223510378, 8)
('TTTTAAAA:TTTTAAAA', 298, 625.0773652694612, -13.082887292564482, 8)
('TAATATTA:TAATATTA', 296, 611.332744924978, -12.754099420638923, 8)
('ATTTAAAT:ATTTAAAT', 195, 472.0488622754492, -12.751990888788157, 8)
('AAACGTTT:AAACGTTT', 89, 305.1719352351581, -12.374759766064596, 8)
('TGATATCA:TGATATCA', 300, 592.1850101282917, -12.007377655226804, 8)
('CCCTAGGG:CCCTAGGG', 232, 498.0984490896831, -11.923415610357333, 8)
('AATATATT:AA

('TCTAGTCA:TGACTAGA', 127, 166.30078883329233, -3.0476102988353855, 8)
('GAGTATCC:GGATACTC', 112, 149.22035031974863, -3.046991215037936, 8)
('CATTCATC:GATGAATG', 238, 289.87123456048505, -3.04672528616986, 8)
('AAACACCT:AGGTGTTT', 234, 285.47454913271173, -3.0466178715991377, 8)
('CCATAATC:GATTATGG', 452, 521.5258400127466, -3.044563329715412, 8)
('AAGATATA:TATATCTT', 183, 229.0745140801343, -3.0442457219122803, 8)
('AGGCGGAT:ATCCGCCT', 144, 185.43203108367044, -3.042634447312455, 8)
('ATGGGACA:TGTCCCAT', 112, 149.14920184573174, -3.0418920058039864, 8)
('CCACTAGC:GCTAGTGG', 192, 239.00545869537925, -3.040549166264462, 8)
('GAATACTC:GAGTATTC', 111, 147.98022138801608, -3.0399917433458126, 8)
('CGTACCCC:GGGGTACG', 72, 102.81105765175714, -3.0387150854799856, 8)
('CGGGGAAG:CTTCCCCG', 245, 297.3390397576638, -3.035353626027701, 8)
('AACCATAT:ATATGGTT', 250, 302.81508315077457, -3.035141219094244, 8)
('AATTGAGT:ACTCAATT', 398, 463.3248031496064, -3.034940324246597, 8)
('CCGTTTAG:CTAAACGG'

('ATGAGCAT:ATGCTCAT', 124, 151.99955037246175, -2.271092211323397, 8)
('AGCAACTG:CAGTTGCT', 232, 269.25455572058087, -2.2704213996021485, 8)
('CAAGCATC:GATGCTTG', 181, 214.2273981867168, -2.2702081689672284, 8)
('CGTTCAAG:CTTGAACG', 92, 116.50200487538734, -2.2700668622569484, 8)
('CGGTGTGC:GCACACCG', 41, 58.335989974339505, -2.269771962363637, 8)
('AATTGTTA:TAACAATT', 474, 526.0573175519381, -2.2697711977513313, 8)
('CGCCTGAA:TTCAGGCG', 191, 225.04698806400194, -2.2695998543329146, 8)
('CTAAAGTA:TACTTTAG', 191, 225.04274056537966, -2.269338127570031, 8)
('GAAACCCC:GGGGTTTC', 459, 510.24412337001723, -2.268670294560692, 8)
('GTCGTCCA:TGGACGAC', 57, 76.88850174216029, -2.2681607357706706, 8)
('ATAGAAGA:TCTTCTAT', 202, 236.9057398897369, -2.267862055535011, 8)
('AGGAAGTA:TACTTCCT', 136, 165.1411904724654, -2.267696170204318, 8)
('GAGGAGGC:GCCTCCTC', 124, 151.95200216069824, -2.267590184635454, 8)
('GATAGGCC:GGCCTATC', 24, 37.97237005911146, -2.2674485519771124, 8)
('GGTAATTA:TAATTACC', 4

('GAATTATA:TATAATTC', 243, 272.03459887070113, -1.760404461663202, 8)
('CAAAAACC:GGTTTTTG', 648, 694.3631897536558, -1.7595522792985012, 8)
('GAAACGGC:GCCGTTTC', 223, 250.86147928064338, -1.759118947386416, 8)
('GTGCCGTA:TACGGCAC', 32, 43.615356422943016, -1.75879095347408, 8)
('CCCCTTCG:CGAAGGGG', 107, 126.80347477784247, -1.7586518712139019, 8)
('TGATTAAA:TTTAATCA', 843, 895.6221053490693, -1.7584679958222225, 8)
('GGGATCCC:GGGATCCC', 6, 12.122448979591839, -1.7584514412115093, 8)
('AGGCCACG:CGTGGCCT', 8, 14.754312270980426, -1.7584180047286444, 8)
('CCTTTATA:TATAAAGG', 184, 209.4457299428322, -1.758271182310302, 8)
('CACAACTG:CAGTTGTG', 224, 251.90506961929393, -1.7582179700598914, 8)
('AGCAGGAG:CTCCTGCT', 148, 170.98516621634337, -1.7578179733859012, 8)
('AAGCGTAG:CTACGCTT', 33, 44.75990039117636, -1.7577638123919792, 8)
('AAGGTACA:TGTACCTT', 143, 165.6175171863079, -1.7575071152578126, 8)
('TCAGAGCA:TGCTCTGA', 129, 150.55965704231448, -1.7570833191794217, 8)
('TGATCGCA:TGCGATCA', 

('GAATAAAA:TTTTATTC', 300, 323.32177459513395, -1.297043254095478, 8)
('ATGGCCGG:CCGGCCAT', 69, 80.64733129966775, -1.2969821773610994, 8)
('CCAGCAAG:CTTGCTGG', 159, 176.21563720265164, -1.2969000780440698, 8)
('TTCTTAAA:TTTAAGAA', 241, 261.9898669763826, -1.2968084141506393, 8)
('CGGTAGAG:CTCTACCG', 119, 134.0050038181283, -1.296223495057408, 8)
('CTGAGGAA:TTCCTCAG', 307, 330.56581075425487, -1.2961760515726048, 8)
('GTAAGCCA:TGGCTTAC', 170, 187.7594124359444, -1.29608576430839, 8)
('CCATTGTA:TACAATGG', 229, 249.4702789625433, -1.2960517265470817, 8)
('CTCGGGCC:GGCCCGAG', 1, 3.383808095952024, -1.2958911861239981, 8)
('ATCGAAAT:ATTTCGAT', 243, 264.051422145148, -1.2955245028664493, 8)
('ATCTTGGG:CCCAAGAT', 314, 337.80979052675883, -1.2954786633569069, 8)
('GGGAGTGA:TCACTCCC', 179, 197.19011850270573, -1.295386188123652, 8)
('GCAGCTCC:GGAGCTGC', 33, 41.32717076184593, -1.2953315846824218, 8)
('ATATCGAG:CTCGATAT', 217, 236.93794359650775, -1.2953008034086972, 8)
('GTTAGTCA:TGACTAAC', 20

...

('CTCCGGTC:GACCGGAG', 150, 129.532199255331, 1.7984006643071597, 8)
('GATGCAAA:TTTGCATC', 272, 243.9076436409376, 1.7988016027431475, 8)
('ACACTGGG:CCCAGTGT', 139, 119.34832847535458, 1.798850799843937, 8)
('TGAGAGCA:TGCTCTCA', 145, 124.89469338040236, 1.7990478013225513, 8)
('ACCCTCAA:TTGAGGGT', 591, 548.8498572541946, 1.7992430308334715, 8)
('AGAGCGGG:CCCGCTCT', 64, 51.132580095960385, 1.7994685285151515, 8)
('ATGCGTGG:CCACGCAT', 38, 28.407624633431084, 1.7997393548251992, 8)
('CATTGACG:CGTCAATG', 216, 191.11637499758197, 1.7999929814198958, 8)
('AATGAAAG:CTTTCATT', 242, 215.57120929044748, 1.8000675733276914, 8)
('CTTGTGTA:TACACAAG', 83, 68.13953399331804, 1.8002591572654532, 8)
('ATTTTGCC:GGCAAAAT', 690, 644.3001198428727, 1.8004935331621243, 8)
('AAAGGGGT:ACCCCTTT', 369, 335.996625726649, 1.8005352765694556, 8)
('ACTCCACG:CGTGGAGT', 75, 60.94314163888894, 1.800643754621747, 8)
('ATTCAAAC:GTTTGAAT', 339, 307.422754491018, 1.8010097874422812, 8)
('ACCGAGCC:GGCTCGGT', 113, 95.4050174

('AAGATGCG:CGCATCTT', 324, 286.15786339362023, 2.2370822949165126, 8)
('AGTCCTGA:TCAGGACT', 326, 288.0328420941562, 2.237156145733174, 8)
('CATATGCG:CGCATATG', 25, 16.03915290008005, 2.23747847635796, 8)
('ATCGGTGC:GCACCGAT', 175, 147.79426822890383, 2.2378776750742344, 8)
('CGTAATAC:GTATTACG', 84, 65.83993171319543, 2.238076347097666, 8)
('AGAGAAAG:CTTTCTCT', 157, 131.34990398571506, 2.238093968923946, 8)
('GACCTGAA:TTCAGGTC', 267, 232.84855692511331, 2.238101264814891, 8)
('ACGAGGTA:TACCTCGT', 102, 81.7623979549765, 2.2381276727767343, 8)
('TCACCAGA:TCTGGTGA', 362, 321.84491189540694, 2.238347725119957, 8)
('GGGCAACC:GGTTGCCC', 195, 166.14741315594978, 2.2384287732934802, 8)
('TGCGGCCA:TGGCCGCA', 99, 79.09191176470588, 2.2385461400707722, 8)
('AAATGCCG:CGGCATTT', 307, 270.2022576876254, 2.23864552041336, 8)
('CCCCTCGG:CCGAGGGG', 173, 145.95158561800164, 2.23893801944998, 8)
('CGGGCTCA:TGAGCCCG', 64, 48.420095799152136, 2.2389969892771857, 8)
('TCACCCCA:TGGGGTGA', 570, 518.99290747345

('CGTCGCAG:CTGCGACG', 142, 110.44689034029454, 3.0024027792867107, 8)
('AAACGCAA:TTGCGTTT', 196, 158.23227842820992, 3.002467440504764, 8)
('ATATGTGA:TCACATAT', 104, 77.55365900902648, 3.003080179928857, 8)
('AGGCTAAC:GTTAGCCT', 224, 183.33199995877226, 3.0035823368868684, 8)
('AAAATGTC:GACATTTT', 413, 356.28743407489986, 3.0046261420223863, 8)
('ACGTCAGC:GCTGACGT', 54, 35.9767644054991, 3.004850430555415, 8)
('CCCAAAAC:GTTTTGGG', 988, 897.9617869650231, 3.0048767615184326, 8)
('AAATGGTC:GACCATTT', 241, 198.6425618841625, 3.0053834147223513, 8)
('TATACGAA:TTCGTATA', 57, 38.37956253019475, 3.0056686237080803, 8)
('ACAGTAGA:TCTACTGT', 226, 185.10710157465286, 3.0056767692811444, 8)
('GACGCAGA:TCTGCGTC', 131, 100.81775972306028, 3.005980616173087, 8)
('TATTCTCA:TGAGAATA', 412, 355.33544727730174, 3.006100978203156, 8)
('AATCTATC:GATAGATT', 289, 242.21519214787864, 3.0061587316841947, 8)
('AACATCAC:GTGATGTT', 277, 231.2721767843536, 3.006950176234509, 8)
('CAGGGTGA:TCACCCTG', 296, 248.5762

('TCCCGCCA:TGGCGGGA', 410, 309.26138089461114, 5.728528008599062, 8)
('ACTCAACC:GGTTGAGT', 450, 343.7738223750033, 5.729360501938033, 8)
('CACTGTTC:GAACAGTG', 232, 159.59995285242812, 5.730962609074836, 8)
('TAATCCTA:TAGGATTA', 562, 441.520812680224, 5.733905919934052, 8)
('CTGCGATA:TATCGCAG', 105, 60.41204860988317, 5.736640965015449, 8)
('CCGGAAAA:TTTTCCGG', 628, 499.735625920976, 5.737885486777376, 8)
('ATTCATTG:CAATGAAT', 362, 267.9801545257462, 5.743502160050024, 8)
('ATTGGCTC:GAGCCAAT', 280, 198.955980531595, 5.745777935941546, 8)
('ACAAATGG:CCATTTGT', 387, 289.2602101880544, 5.746935263349442, 8)
('AGGGGAAA:TTTCCCCT', 827, 677.1931236318775, 5.757012094778121, 8)
('CCGGTGGA:TCCACCGG', 335, 244.89053834853058, 5.758272665730026, 8)
('AAACCTAA:TTAGGTTT', 399, 299.3700635573671, 5.7583126966265565, 8)
('CAGGGACA:TGTCCCTG', 243, 168.29684955441596, 5.758459384683433, 8)
('GATTCGAA:TTCGAATC', 27, 9.370162590311846, 5.759370842433017, 8)
('AACATTTG:CAAATGTT', 388, 289.7815884146277, 5

"\nExpectation = Prob * Genome Length\n\nProb = Pr(GCGATCGC) = (1/N) * (count(GCGATCG) * count(CGATCGC))/count(CGATCG)\n\nOR if kmer's reverse complement is different\n\nProb = Pr(CGATCGCC) + Pr(GGCGATCG)\n\nZ = (observed - expectation)/sd\n\nsd = sqrt(N*P * (1-P))\n"

In [58]:
s = sfm.prDict[8]['GCGATCGC'] +sfm.prDict[8][revComp('GCGATCGC')]


0.004677479412691261

In [71]:
k = 8
kmer = 'GCGATCGC'

num = sfm.ctDict[k-1][kmer[:-1]] * sfm.ctDict[k-1][kmer[1:]]
den = sfm.ctDict[k-2][kmer[1:-1]]*
pr = num/den
print(f'pr = {pr}')

pr = 7938.1912392232


In [67]:
numm = sfm.ctDict[7]['GCGATCG'] * sfm.ctDict[7]['CGATCGC']
denn = sfm.ctDict[6]['CGATCG'] * sfm.N
prr = numm/denn
print(f'prr = {prr}')


prr = 0.0011693698531728151


In [8]:
ct = sfm.ctDict[8]['GCGATCGC']
print(ct)

7683


In [77]:
sfm.ctDict[7]['CGATCGC']

9570

In [78]:
sfm.ctDict[7][revComp('GCGATCG')]

9570

In [79]:
sfm.ctDict[7][revComp('CGATCGC')]

9525